### Installations and importing required modules

In [1]:
!pip install transformers==2.8.0
!pip install torch==1.4.0
!pip install -U spacy
!pip3 install SpeechRecognition pydub
!python -m spacy download en_core_web_sm
!pip install plac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 21.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# importing libraries 

import os 
import json 
import plac
import spacy
import torch
import random
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from pydub import AudioSegment
import speech_recognition as sr 
from sklearn.svm import LinearSVC
from spacy.training import Example
from pydub.silence import split_on_silence
from __future__ import unicode_literals, print_function
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


### 1. Speech to Text

In [5]:
filename = "/content/sales_call_telephone_marketers.wav"

In [6]:
# create a speech recognition object
r = sr.Recognizer()

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 1000 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        min_silence_len = 1000,
        silence_thresh = sound.dBFS-14,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # directory to store the audio chunks (if not exist, create one)
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `audio-chunks` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [7]:
# min scielence = 1000, keep scilence = 500
task_1_output = get_large_audio_transcription('/content/sales_call_telephone_marketers.wav')
print('\n')
task_1_output = task_1_output.lower()
task_1_output

audio-chunks/chunk1.wav : Hello hi nancy this is mike from at&t incorporation. 
audio-chunks/chunk2.wav : Using our prepaid connection for a couple of years now right. 
audio-chunks/chunk3.wav : How would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. 
audio-chunks/chunk4.wav : I would love that. 
audio-chunks/chunk5.wav : There will be a monthly rental which you will have to pay like any other postpaid connection. 




'hello hi nancy this is mike from at&t incorporation. using our prepaid connection for a couple of years now right. how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. i would love that. there will be a monthly rental which you will have to pay like any other postpaid connection. '

In [8]:
# min silence len = 1000, keep scielence = 250
task_1_output_2 = get_large_audio_transcription('/content/sales_call_telephone_marketers.wav')
print('\n')
task_1_output_2

audio-chunks/chunk1.wav : Hello hi nancy this is mike from at&t incorporation. 
audio-chunks/chunk2.wav : Using our prepaid connection for a couple of years now right. 
audio-chunks/chunk3.wav : How would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. 
audio-chunks/chunk4.wav : I would love that. 
audio-chunks/chunk5.wav : There will be a monthly rental which you will have to pay like any other postpaid connection. 




'Hello hi nancy this is mike from at&t incorporation. Using our prepaid connection for a couple of years now right. How would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. I would love that. There will be a monthly rental which you will have to pay like any other postpaid connection. '

### Task 2: Train an NLU model to classify intents and recognize entities. 
- 2.1 Classify intents : classification Task

- 2.2 Recognize entities : Named Entity Recognition

#### 2.1 Intent Classification using SVM
- It is just a binary classification task.
- Since it only has 2 classes: 1. Intro, 2. Purpose

In [9]:
# Data for Task 2.1

x = [
     'My name is Jeff and I am calling from Amazon.', 
     'I am calling about your Microsoft Azure subscription.', 
     'I am calling from Microsoft and myname is Satya.',
     'This is a call regarding your Google Cloud Platform account.',
     'I am Sundar and this is call from Google.',
     'I would like to talk about your Amazon Web Services account.',
     'This is Ankitha calling from Amazon.',
     'I am calling regarding your jio postpaid connection.',
     'Hello Sravani this is Arjun calling from Zomato.',
     'You have been using our service from couple of years now',
     'Good morning madam this is Ram calling from Jio.',
     'How would you rate our service?'
]
y = [
     'Intro',
     'Purpose',
     'Intro',
     'Purpose',
     'Intro',
     'Purpose',
     'Intro',
     'Purpose',
     'Intro',
     'Purpose',
     'Intro',
     'Purpose'
]

In [10]:
data = pd.DataFrame(list(zip(x, y)),
               columns =['Sentence', 'intent'])

data

,Sentence,intent
0,My name is Jeff and I am calling from Amazon.,Intro
1,I am calling about your Microsoft Azure subscr...,Purpose
2,I am calling from Microsoft and myname is Satya.,Intro
3,This is a call regarding your Google Cloud Pla...,Purpose
4,I am Sundar and this is call from Google.,Intro
5,I would like to talk about your Amazon Web Ser...,Purpose
6,This is Ankitha calling from Amazon.,Intro
7,I am calling regarding your jio postpaid conne...,Purpose
8,Hello Sravani this is Arjun calling from Zomato.,Intro
9,You have been using our service from couple of...,Purpose


In [11]:
# Create SVM classifier to classify the text

vectorizer = CountVectorizer()
svm = LinearSVC()
X_train, Y_train = data.Sentence.copy(), data.intent.copy()
X_train = vectorizer.fit_transform(data.Sentence)
_ = svm.fit(X_train, Y_train)

In [12]:
# Testing the model with the output generated in Task - 1.
X_test = task_1_output.split('. ')

X_test =  vectorizer.transform(X_test)

y_pred = svm.predict(X_test)

In [13]:
print(y_pred)

['Intro' 'Purpose' 'Purpose' 'Purpose' 'Purpose' 'Purpose']


#### 2.2 Named Entity Recognition
- We need to train the NER using our data since we need to add some entities like product.

In [14]:
# Creating the data for 2.1 based on the examples given in Task - 2

TRAIN_DATA = [
    ('my name is jeff and I am calling from amazon.', {
        'entities': [(11, 15, 'person'), (38, 44, 'org')]
    }),
    ('I want to talk to you about your postpaid connection.', {
        'entities': [(33, 52, 'product')]
    }),
     ('i am calling from microsoft and my name is satya.', {
        'entities': [(18, 27, 'org'), (43, 48, 'person')]
    }),
     ('i am sundar and this is call from google.', {
        'entities': [(5, 11, 'person'), (34, 40, 'org')]
    }),
    ('hello mike i am nancy.', {
        'entities': [(6, 10, 'person'), (16, 21, 'person')]
    }),
     ('i am calling about your microsoft azure subscription.', {
        'entities': [(24, 39, 'product')]
    }),
    ('How about a prepaid connection.', {
        'entities': [(12, 30, 'product')]
    }),
     ('this is a call regarding your google cloud platform account.', {
        'entities': [(30, 51, 'product')]
    }),
    ('I would like to talk about your amazon web services account.', {
        'entities': [(32, 51, 'product')]
    })
]

In [15]:
# Creating a blank model
model = None
output_dir=Path("/content/ner")
n_iter=100

In [16]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [17]:
# Training the NER using the data
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example]) 
        print(losses)

100%|██████████| 9/9 [00:00<00:00, 42.36it/s]


{}


100%|██████████| 9/9 [00:00<00:00, 42.96it/s]


{}


100%|██████████| 9/9 [00:00<00:00, 44.93it/s]


...

{}


100%|██████████| 9/9 [00:00<00:00, 44.11it/s]


{}


100%|██████████| 9/9 [00:00<00:00, 46.90it/s]


{}


In [18]:
# Testing the model with the output obtained in Task - 1
for x in task_1_output_2.split('. '):
  doc = nlp(x.lower())
  print(x)
  print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Hello hi nancy this is mike from at&t incorporation
Entities [('nancy', 'person'), ('mike', 'person'), ('at&t incorporation', 'product')]
Using our prepaid connection for a couple of years now right
Entities [('prepaid connection', 'product'), ('couple of years', 'product')]
How would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers
Entities [('postpaid connections', 'product')]
I would love that
Entities []
There will be a monthly rental which you will have to pay like any other postpaid connection
Entities [('there', 'org'), ('monthly rental which', 'product'), ('pay', 'person'), ('postpaid connection', 'product')]

Entities []


In [19]:
# Saving the trained model
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/ner


### Task 3: 
- Separate the sentences in the output of task 1. On each sentence, apply the
model trained in task 2 to classify its intent and recognize the entities present in it.

In [20]:
# Printing the output of Task - 3
task_1_output = task_1_output.strip()
x_test = task_1_output.split('. ')
length = len(x_test)
X_test =  vectorizer.transform(x_test)
y_pred = svm.predict(X_test)
for text in range(length):
  print('||______________________________________________________________________________________________________________________________')
  print('|| Text  :   ', x_test[text], '\n|| Intent : ', y_pred[text] )
  doc = nlp(x_test[text].lower())
  print('|| Entities : ', [(ent.text, ent.label_) for ent in doc.ents])
  print('||______________________________________________________________________________________________________________________________')

||______________________________________________________________________________________________________________________________
|| Text  :    hello hi nancy this is mike from at&t incorporation 
|| Intent :  Intro
|| Entities [('nancy', 'person'), ('mike', 'person'), ('at&t incorporation', 'product')]
||______________________________________________________________________________________________________________________________
||______________________________________________________________________________________________________________________________
|| Text  :    using our prepaid connection for a couple of years now right 
|| Intent :  Purpose
|| Entities [('prepaid connection', 'product'), ('couple of years', 'product')]
||______________________________________________________________________________________________________________________________
||______________________________________________________________________________________________________________________________
|| 

In [32]:
task_3_output = []

for text in range(length):
  a = {}
  doc = nlp(x_test[text].lower())
  a['sentence'] = x_test[text]
  a['entities'] = [(ent.text, ent.label_) for ent in doc.ents]
  a['intent'] = y_pred[text]
  task_3_output.append(a)

In [33]:
task_3_output

[{'entities': [('nancy', 'person'),
   ('mike', 'person'),
   ('at&t incorporation', 'product')],
  'intent': 'Intro',
  'sentence': 'hello hi nancy this is mike from at&t incorporation'},
 {'entities': [('prepaid connection', 'product'),
   ('couple of years', 'product')],
  'intent': 'Purpose',
  'sentence': 'using our prepaid connection for a couple of years now right'},
 {'entities': [('postpaid connections', 'product')],
  'intent': 'Purpose',
  'sentence': 'how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers'},
 {'entities': [], 'intent': 'Purpose', 'sentence': 'i would love that'},
 {'entities': [('there', 'org'),
   ('monthly rental which', 'product'),
   ('pay', 'person'),
   ('postpaid connection', 'product')],
  'intent': 'Purpose',
  'sentence': 'there will be a monthly rental which you will have to pay like any other postpaid connection.'}]

In [34]:
exact_output = {
    "task_1_output":task_1_output,
    "task_3_output":task_3_output
}

In [35]:
exact_output

{'task_1_output': 'hello hi nancy this is mike from at&t incorporation. using our prepaid connection for a couple of years now right. how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. i would love that. there will be a monthly rental which you will have to pay like any other postpaid connection.',
 'task_3_output': [{'entities': [('nancy', 'person'),
    ('mike', 'person'),
    ('at&t incorporation', 'product')],
   'intent': 'Intro',
   'sentence': 'hello hi nancy this is mike from at&t incorporation'},
  {'entities': [('prepaid connection', 'product'),
    ('couple of years', 'product')],
   'intent': 'Purpose',
   'sentence': 'using our prepaid connection for a couple of years now right'},
  {'entities': [('postpaid connections', 'product')],
   'intent': 'Purpose',
   'sentence': 'how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers'},
  {'entities': [], 'intent': 'Pu

In [38]:
json_object  = json.dumps(exact_output, indent = 4) 

In [39]:
json_object

'{\n"task_1_output": "hello hi nancy this is mike from at&t incorporation. using our prepaid connection for a couple of years now right. how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. i would love that. there will be a monthly rental which you will have to pay like any other postpaid connection.",\n"task_3_output": [\n{\n"sentence": "hello hi nancy this is mike from at&t incorporation",\n"entities": [\n[\n"nancy",\n"person"\n],\n[\n"mike",\n"person"\n],\n[\n"at&t incorporation",\n"product"\n]\n],\n"intent": "Intro"\n},\n{\n"sentence": "using our prepaid connection for a couple of years now right",\n"entities": [\n[\n"prepaid connection",\n"product"\n],\n[\n"couple of years",\n"product"\n]\n],\n"intent": "Purpose"\n},\n{\n"sentence": "how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers",\n"entities": [\n[\n"postpaid connections",\n"product"\n]\n],\n"intent": "Purpose"\

In [40]:
out_file = open("myfile.json", "w") 
json.dump(json_object, out_file, indent = 4) 
out_file.close() 

### Task 4: For the whole text generated from the audio file generate a summary report. 

In [22]:

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

text = task_1_output

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)


original text preprocessed: 
 hello hi nancy this is mike from at&t incorporation. using our prepaid connection for a couple of years now right. how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. i would love that. there will be a monthly rental which you will have to pay like any other postpaid connection.


Summarized text: 
 at&t incorporation uses our prepaid connection for a couple of years. it allows you to make unlimited unlimited voice call to 318ti numbers. you will have to pay monthly rental like any other postpaid connection.


In [24]:
task_1_output

'hello hi nancy this is mike from at&t incorporation. using our prepaid connection for a couple of years now right. how would you like a postpaid connections that allows you to make free unlimited voice call to 318ti numbers. i would love that. there will be a monthly rental which you will have to pay like any other postpaid connection.'

In [23]:
output

'at&t incorporation uses our prepaid connection for a couple of years. it allows you to make unlimited unlimited voice call to 318ti numbers. you will have to pay monthly rental like any other postpaid connection.'